In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score, classification_report, confusion_matrix
import missingno as msno
from sklearn.preprocessing import OneHotEncoder
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
import warnings
warnings.simplefilter("ignore")

In [7]:
# Load Train Dataset

Train = pd.read_csv("../archive/Train-1542865627584.csv")
Train_Inpatientdata=pd.read_csv("../archive/Train_Inpatientdata-1542865627584.csv")
Train_Outpatientdata=pd.read_csv("../archive/Train_Outpatientdata-1542865627584.csv")
Train_Beneficiarydata=pd.read_csv("../archive/Train_Beneficiarydata-1542865627584.csv")

# Load Test Dataset
Test = pd.read_csv("../archive/Test-1542969243754.csv")
Test_Beneficiarydata=pd.read_csv("../archive/Test_Beneficiarydata-1542969243754.csv")
Test_Inpatientdata=pd.read_csv("../archive/Test_Inpatientdata-1542969243754.csv")
Test_Outpatientdata=pd.read_csv("../archive/Test_Outpatientdata-1542969243754.csv")

In [8]:
# Shape of data

print("The Training inpatient data has {} records and {} fields. \n" .format(Train_Inpatientdata.shape[0], Train_Inpatientdata.shape[1]))
print("The Training outpatient data has {} records and {} fields. \n" .format(Train_Outpatientdata.shape[0], Train_Outpatientdata.shape[1]))
print("The Training Benficiary data has {} records and {} fields. \n" .format(Train_Beneficiarydata.shape[0], Train_Beneficiarydata.shape[1]))

print("The Test inpatient data has {} records and {} fields. \n" .format(Test_Inpatientdata.shape[0], Test_Inpatientdata.shape[1]))
print("The Test outpatient data has {} records and {} fields. \n" .format(Test_Outpatientdata.shape[0], Test_Outpatientdata.shape[1]))
print("The Test Benficiary data has {} records and {} fields. \n" .format(Test_Beneficiarydata.shape[0], Test_Beneficiarydata.shape[1]))

The Training inpatient data has 40474 records and 30 fields. 

The Training outpatient data has 517737 records and 27 fields. 

The Training Benficiary data has 138556 records and 25 fields. 

The Test inpatient data has 9551 records and 30 fields. 

The Test outpatient data has 125841 records and 27 fields. 

The Test Benficiary data has 63968 records and 25 fields. 



In [9]:
Train_Beneficiarydata.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,2,2,2,2,2,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,1,2,2,2,2,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,2,2,2,2,1,2,1,2,2,2,2,0,0,1790,1200


In [10]:
print("The Beneficiary Train Data has {} records and {} fields. \n" .format(Train_Beneficiarydata.shape[0], Train_Beneficiarydata.shape[1]))
print("The Beneficiary Test Data has {} records and {} fields. \n" .format(Test_Beneficiarydata.shape[0], Test_Beneficiarydata.shape[1]))

The Beneficiary Train Data has 138556 records and 25 fields. 

The Beneficiary Test Data has 63968 records and 25 fields. 



In [11]:
#Lets check missing values in each column in inpatient data
Train_Inpatientdata.isna().sum()

BeneID                        0
ClaimID                       0
ClaimStartDt                  0
ClaimEndDt                    0
Provider                      0
InscClaimAmtReimbursed        0
AttendingPhysician          112
OperatingPhysician        16644
OtherPhysician            35784
AdmissionDt                   0
ClmAdmitDiagnosisCode         0
DeductibleAmtPaid           899
DischargeDt                   0
DiagnosisGroupCode            0
ClmDiagnosisCode_1            0
ClmDiagnosisCode_2          226
ClmDiagnosisCode_3          676
ClmDiagnosisCode_4         1534
ClmDiagnosisCode_5         2894
ClmDiagnosisCode_6         4838
ClmDiagnosisCode_7         7258
ClmDiagnosisCode_8         9942
ClmDiagnosisCode_9        13497
ClmDiagnosisCode_10       36547
ClmProcedureCode_1        17326
ClmProcedureCode_2        35020
ClmProcedureCode_3        39509
ClmProcedureCode_4        40358
ClmProcedureCode_5        40465
ClmProcedureCode_6        40474
dtype: int64

In [12]:
# Check for duplicates

print('There are '+ str(Train_Beneficiarydata.duplicated().sum())+ ' duplicate  in the beneficiary training dataset.')
print('There are '+ str(Test_Beneficiarydata.duplicated().sum())+ ' duplicate in the beneficiary test dataset.')

There are 0 duplicate  in the beneficiary training dataset.
There are 0 duplicate in the beneficiary test dataset.


In [13]:
# Number of unique benificaries

print('They are', Train_Beneficiarydata['BeneID'].nunique(), 'unique beneficiaries in the training data set')
print('They are', Test_Beneficiarydata['BeneID'].nunique(), 'unique beneficiaries in the test data set')

They are 138556 unique beneficiaries in the training data set
They are 63968 unique beneficiaries in the test data set


In [14]:
# Convert 'DOB' and 'DOD' to datetime format
Train_Beneficiarydata['DOB'] = pd.to_datetime(Train_Beneficiarydata['DOB'], format='%Y-%m-%d')
Train_Beneficiarydata['DOD'] = pd.to_datetime(Train_Beneficiarydata['DOD'], format='%Y-%m-%d', errors='ignore')
Test_Beneficiarydata['DOB'] = pd.to_datetime(Test_Beneficiarydata['DOB'], format='%Y-%m-%d')
Test_Beneficiarydata['DOD'] = pd.to_datetime(Test_Beneficiarydata['DOD'], format='%Y-%m-%d', errors='ignore')

# Calculate age in years and round to the nearest integer
Train_Beneficiarydata['Age'] = round((Train_Beneficiarydata['DOD'] - Train_Beneficiarydata['DOB']).dt.days / 365)
Test_Beneficiarydata['Age'] = round((Test_Beneficiarydata['DOD'] - Test_Beneficiarydata['DOB']).dt.days / 365)

In [15]:
# The final DOD date is '2009-12-01' . We can use that to calculate the ages of those with no DOD

# Convert 'DOB' to datetime format
Train_Beneficiarydata['DOB'] = pd.to_datetime(Train_Beneficiarydata['DOB'], format='%Y-%m-%d')
Test_Beneficiarydata['DOB'] = pd.to_datetime(Test_Beneficiarydata['DOB'], format='%Y-%m-%d')

# Calculate age for beneficiaries with missing 'Age' values, assuming 2009-12-01 as the reference date
Train_Beneficiarydata['Age'].fillna(
    round((pd.to_datetime('2009-12-01') - Train_Beneficiarydata['DOB']).dt.days / 365),
    inplace=True
)
Test_Beneficiarydata['Age'].fillna(
    round((pd.to_datetime('2009-12-01') - Test_Beneficiarydata['DOB']).dt.days / 365),
    inplace=True
)

In [16]:
# Create a new 'AliveorDead' column and flag deceased beneficiaries
Train_Beneficiarydata['AliveorDead'] = Train_Beneficiarydata['DOD'].notna().astype(int)
Test_Beneficiarydata['AliveorDead'] = Test_Beneficiarydata['DOD'].notna().astype(int)

In [17]:
Train_Beneficiarydata.groupby(['AliveorDead'])['BeneID'].nunique()

AliveorDead
0    137135
1      1421
Name: BeneID, dtype: int64

In [18]:
Train_Inpatientdata.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,6186,1068.0,2009-09-02,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,29590,1068.0,2009-09-20,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,431,1068.0,2009-02-22,067,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,78321,1068.0,2009-08-30,975,042,3051,34400,5856,42732,486,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN


In [19]:
#Lets check missing values in each column in inpatient data
Train_Inpatientdata.isna().sum()

BeneID                        0
ClaimID                       0
ClaimStartDt                  0
ClaimEndDt                    0
Provider                      0
InscClaimAmtReimbursed        0
AttendingPhysician          112
OperatingPhysician        16644
OtherPhysician            35784
AdmissionDt                   0
ClmAdmitDiagnosisCode         0
DeductibleAmtPaid           899
DischargeDt                   0
DiagnosisGroupCode            0
ClmDiagnosisCode_1            0
ClmDiagnosisCode_2          226
ClmDiagnosisCode_3          676
ClmDiagnosisCode_4         1534
ClmDiagnosisCode_5         2894
ClmDiagnosisCode_6         4838
ClmDiagnosisCode_7         7258
ClmDiagnosisCode_8         9942
ClmDiagnosisCode_9        13497
ClmDiagnosisCode_10       36547
ClmProcedureCode_1        17326
ClmProcedureCode_2        35020
ClmProcedureCode_3        39509
ClmProcedureCode_4        40358
ClmProcedureCode_5        40465
ClmProcedureCode_6        40474
dtype: int64

In [20]:
# Calculate number of days patient was admitted from AdmissionDt and DischargeDt

# Convert 'AdmissionDt' and 'DischargeDt' to datetime format
Train_Inpatientdata['AdmissionDt'] = pd.to_datetime(Train_Inpatientdata['AdmissionDt'], format='%Y-%m-%d')
Train_Inpatientdata['DischargeDt'] = pd.to_datetime(Train_Inpatientdata['DischargeDt'], format='%Y-%m-%d')
Test_Inpatientdata['AdmissionDt'] = pd.to_datetime(Test_Inpatientdata['AdmissionDt'], format='%Y-%m-%d')
Test_Inpatientdata['DischargeDt'] = pd.to_datetime(Test_Inpatientdata['DischargeDt'], format='%Y-%m-%d')

# Calculate the number of days of admission, adding 1 to include both the admission and discharge days
Train_Inpatientdata['NumberofDaysAdmitted'] = (Train_Inpatientdata['DischargeDt'] - Train_Inpatientdata['AdmissionDt']).dt.days + 1
Test_Inpatientdata['NumberofDaysAdmitted'] = (Test_Inpatientdata['DischargeDt'] - Test_Inpatientdata['AdmissionDt']).dt.days + 1

In [21]:
# Convert 'ClaimStartDt' and 'ClaimEndDt' to datetime format
Train_Inpatientdata['ClaimEndDt'] = pd.to_datetime(Train_Inpatientdata['ClaimEndDt'], format='%Y-%m-%d')
Train_Inpatientdata['ClaimStartDt'] = pd.to_datetime(Train_Inpatientdata['ClaimStartDt'], format='%Y-%m-%d')

Test_Inpatientdata['ClaimEndDt'] = pd.to_datetime(Test_Inpatientdata['ClaimEndDt'], format='%Y-%m-%d')
Test_Inpatientdata['ClaimStartDt'] = pd.to_datetime(Test_Inpatientdata['ClaimStartDt'], format='%Y-%m-%d')

# Duration of Claim
Train_Inpatientdata['DurationofClaim'] = (Train_Inpatientdata['ClaimEndDt'] - Train_Inpatientdata['ClaimStartDt']).dt.days
Test_Inpatientdata['DurationofClaim'] = (Test_Inpatientdata['ClaimEndDt'] - Test_Inpatientdata['ClaimStartDt']).dt.days

In [22]:
# Add column admitted

Train_Inpatientdata['Admitted'] =1
Test_Inpatientdata['Admitted'] =1

In [23]:
Train_Outpatientdata.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,V5866,V1272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,NaN,NaN,6115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,NaN,NaN,2723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,30000,72887,4280,7197,V4577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947


In [24]:
# Check for duplicate rows 
print('Duplicate rows in train set', Train_Outpatientdata.duplicated().sum())
print('Duplicate rows in test set', Test_Outpatientdata.duplicated().sum())

Duplicate rows in train set 0
Duplicate rows in test set 0


In [25]:
Train_Outpatientdata.isnull().sum()

BeneID                         0
ClaimID                        0
ClaimStartDt                   0
ClaimEndDt                     0
Provider                       0
InscClaimAmtReimbursed         0
AttendingPhysician          1396
OperatingPhysician        427120
OtherPhysician            322691
ClmDiagnosisCode_1         10453
ClmDiagnosisCode_2        195380
ClmDiagnosisCode_3        314480
ClmDiagnosisCode_4        392141
ClmDiagnosisCode_5        443393
ClmDiagnosisCode_6        468981
ClmDiagnosisCode_7        484776
ClmDiagnosisCode_8        494825
ClmDiagnosisCode_9        502899
ClmDiagnosisCode_10       516654
ClmProcedureCode_1        517575
ClmProcedureCode_2        517701
ClmProcedureCode_3        517733
ClmProcedureCode_4        517735
ClmProcedureCode_5        517737
ClmProcedureCode_6        517737
DeductibleAmtPaid              0
ClmAdmitDiagnosisCode     412312
dtype: int64

In [26]:
# Add column admitted

Train_Outpatientdata['Admitted'] = 0
Test_Outpatientdata['Admitted'] = 0

In [27]:
# Convert 'ClaimStartDt' and 'ClaimEndDt' to datetime format
Train_Outpatientdata['ClaimEndDt'] = pd.to_datetime(Train_Outpatientdata['ClaimEndDt'], format='%Y-%m-%d')
Train_Outpatientdata['ClaimStartDt'] = pd.to_datetime(Train_Outpatientdata['ClaimStartDt'], format='%Y-%m-%d')

Test_Outpatientdata['ClaimEndDt'] = pd.to_datetime(Test_Outpatientdata['ClaimEndDt'], format='%Y-%m-%d')
Test_Outpatientdata['ClaimStartDt'] = pd.to_datetime(Test_Outpatientdata['ClaimStartDt'], format='%Y-%m-%d')

# Duration of Claim
Train_Outpatientdata['DurationofClaim'] = (Train_Outpatientdata['ClaimEndDt'] - Train_Outpatientdata['ClaimStartDt']).dt.days
Test_Outpatientdata['DurationofClaim'] = (Test_Outpatientdata['ClaimEndDt'] - Test_Outpatientdata['ClaimStartDt']).dt.days

In [28]:
#Identify common columns between Outpatient and inpatient data

common_cols = list(set(Train_Inpatientdata.columns).intersection(set(Train_Outpatientdata.columns)))
print(common_cols)

['ClmProcedureCode_4', 'ClmDiagnosisCode_8', 'ClmProcedureCode_6', 'ClmProcedureCode_2', 'ClaimEndDt', 'ClmDiagnosisCode_4', 'ClmAdmitDiagnosisCode', 'BeneID', 'ClmDiagnosisCode_10', 'OperatingPhysician', 'Admitted', 'ClmProcedureCode_3', 'ClaimStartDt', 'DeductibleAmtPaid', 'ClmDiagnosisCode_9', 'ClmDiagnosisCode_3', 'DurationofClaim', 'ClmProcedureCode_1', 'ClmDiagnosisCode_5', 'Provider', 'InscClaimAmtReimbursed', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_6', 'OtherPhysician', 'ClmProcedureCode_5', 'ClaimID', 'AttendingPhysician']


In [29]:
# Merge the DataFrames on the common columns, using an outer join

Train_Allpatientdata = pd.merge(Train_Outpatientdata, Train_Inpatientdata, on=common_cols, how='outer')
Test_Allpatientdata = pd.merge(Test_Outpatientdata, Test_Inpatientdata, on=common_cols, how='outer')

print(Train_Allpatientdata.shape)
print(Test_Allpatientdata.shape)

(558211, 33)
(135392, 33)


In [30]:
df_train = Train_Allpatientdata.merge(Train_Beneficiarydata, on='BeneID', how='inner')

df_test = Test_Allpatientdata.merge(Test_Beneficiarydata, on='BeneID', how='inner')

In [31]:
# shape of dataset
print('Training data shape: ', df_train.shape)
print('Test data shape: ', df_test.shape)

Training data shape:  (558211, 59)
Test data shape:  (135392, 59)


In [32]:
# Lets merge patient data with fradulent providers details data with "Provider" as joining key for inner join

df_train1 = pd.merge(Train, df_train,on='Provider')

df_test1 = pd.merge(Test, df_test,on='Provider')

In [33]:
# this value indicates whether the beneficiary has renal disease
df_train1['RenalDiseaseIndicator'].replace('Y','1',inplace=True)
df_train1['RenalDiseaseIndicator'] = df_train1['RenalDiseaseIndicator'].astype(int)
    
    
df_test1['RenalDiseaseIndicator'].replace('Y','1',inplace=True)
df_test1['RenalDiseaseIndicator'] = df_test1['RenalDiseaseIndicator'].astype(int)


# Drop DOD and DOB

df_train1.drop(columns=['DOB', 'DOD'], axis=1, inplace=True)
df_test1.drop(columns=['DOB', 'DOD'], axis=1, inplace=True)

In [34]:
# ClaimDiagnoseIndex, ClmProcedureIndex, ChronicCondIndex

df_train1['ClmDiagnosisCodeIndex'] = df_train1.filter(regex='ClmDiagnosisCode_').notnull().sum(axis=1)
df_test1['ClmDiagnosisCodeIndex'] = df_test1.filter(regex='ClmDiagnosisCode_').notnull().sum(axis=1)

df_train1['ClmProcedureCodeIndex'] = df_train1.filter(regex='ClmProcedureCode_').notnull().sum(axis=1)
df_test1['ClmProcedureCodeIndex'] = df_test1.filter(regex='ClmProcedureCode_').notnull().sum(axis=1)

In [35]:
# drop all features starting 

columns_to_drop = df_train1.filter(regex='ClmProcedureCode_|ClmDiagnosisCode_').columns
df_train1 = df_train1.drop(columns_to_drop, axis=1)
df_test1 = df_test1.drop(columns_to_drop, axis=1)

In [36]:
# fill nan for NumberofDaysAdmitted with 0

df_train1['NumberofDaysAdmitted'] = df_train1['NumberofDaysAdmitted'].fillna(0)
df_test1['NumberofDaysAdmitted'] = df_test1['NumberofDaysAdmitted'].fillna(0)

In [37]:
df_train1 = df_train1.dropna(subset=['AttendingPhysician'])
df_test1 = df_test1.dropna(subset=['AttendingPhysician'])

# Fill missing values with mean

df_train1['DeductibleAmtPaid'] = df_train1['DeductibleAmtPaid'].fillna(df_train1['DeductibleAmtPaid'].mean())
df_test1['DeductibleAmtPaid'] = df_test1['DeductibleAmtPaid'].fillna(df_test1['DeductibleAmtPaid'].mean())

In [38]:
# . Average features grouped by Provider

columns_to_transform = ["InscClaimAmtReimbursed", "DeductibleAmtPaid", "IPAnnualReimbursementAmt", "IPAnnualDeductibleAmt",
    "OPAnnualReimbursementAmt", "OPAnnualDeductibleAmt", "Age", "NoOfMonths_PartACov", "NoOfMonths_PartBCov","DurationofClaim",
                        "NumberofDaysAdmitted"
]

for column in columns_to_transform:
    df_train1[f"PerProviderAvg_{column}"] = df_train1.groupby('Provider')[column].transform('mean')
    df_test1[f"PerProviderAvg_{column}"] = df_test1.groupby('Provider')[column].transform('mean')


In [39]:
# Average features grouped by BeneID
# Average features grouped by Operating Physician


columns_to_transform = [
    "InscClaimAmtReimbursed",
    "DeductibleAmtPaid",
    "IPAnnualReimbursementAmt",
    "IPAnnualDeductibleAmt",
    "OPAnnualReimbursementAmt",
    "OPAnnualDeductibleAmt", 
    "DurationofClaim",
    "NumberofDaysAdmitted"
    
]


for column in columns_to_transform:
  
    
    df_train1[f"PerBeneIDAvg_{column}"] = df_train1.groupby('BeneID')[column].transform('mean')
    df_test1[f"PerBeneIDAvg_{column}"] = df_test1.groupby('BeneID')[column].transform('mean')
    
    df_train1[f"PerAttendingPhysician Avg_{column}"] = df_train1.groupby('AttendingPhysician')[column].transform('mean')
    df_test1[f"PerAttendingPhysician Avg_{column}"] = df_test1.groupby('AttendingPhysician')[column].transform('mean')

In [40]:
# Drop features not needed for model training
df_train1.drop(columns=['ClmAdmitDiagnosisCode', 'Provider', 'State', 'Race', 'Gender', 'County', 'AdmissionDt', 'AttendingPhysician', 'OtherPhysician', 'OperatingPhysician',  
                        'DischargeDt', 'ClaimID', 'ClaimEndDt', 'DiagnosisGroupCode', 'ClaimStartDt', 'BeneID', 'ClaimID'], axis=1, inplace=True)

df_test1.drop(columns=['ClmAdmitDiagnosisCode', 'State', 'Race', 'County', 'Gender', 'AdmissionDt', 'DiagnosisGroupCode', 'OperatingPhysician', 'DischargeDt', 'AttendingPhysician', 'OtherPhysician', 
                       'ClaimID', 'ClaimEndDt', 'ClaimStartDt', 'ClaimID'], axis=1, inplace=True)

In [41]:
# Split data into train and validation set

df_train2 , df_val = train_test_split(df_train1, test_size=0.10, random_state=42)

y_train = df_train2.pop('PotentialFraud')
X_train = df_train2

y_val = df_val.pop('PotentialFraud')
X_val = df_val

X_test = df_test1


print(X_train.shape, X_val.shape, X_test.shape)

(501032, 54) (55671, 54) (135045, 56)


In [42]:
# One hot encode ChronicCond_ features

categorical_cols = [col for col in X_train.columns if col.startswith('ChronicCond_')]

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

encoded_data_train = encoder.fit_transform(X_train[categorical_cols])
encoded_data_test = encoder.fit_transform(X_test[categorical_cols])
encoded_data_val = encoder.fit_transform(X_val[categorical_cols])


encoded_df_train = pd.DataFrame(encoded_data_train, columns=encoder.get_feature_names_out())
encoded_df_test = pd.DataFrame(encoded_data_test, columns=encoder.get_feature_names_out())
encoded_df_val = pd.DataFrame(encoded_data_val, columns=encoder.get_feature_names_out())

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)



X_train = pd.concat([X_train.drop(categorical_cols, axis=1), encoded_df_train], axis=1)
X_test = pd.concat([X_test.drop(categorical_cols, axis=1), encoded_df_test], axis=1)
X_val = pd.concat([X_val.drop(categorical_cols, axis=1), encoded_df_val], axis=1)

In [43]:
# Treating imbalance data in training dataset

from imblearn.over_sampling import SMOTE

from collections import Counter

counter = Counter(y_train)

print('before smoting: ', counter)

smt = SMOTE()

X_train, y_train = smt.fit_resample(X_train, y_train)

counter = Counter(y_train)
print('After smoting: ', counter)

before smoting:  Counter({'No': 310111, 'Yes': 190921})
After smoting:  Counter({'Yes': 310111, 'No': 310111})


In [44]:
X_train.head()

,InscClaimAmtReimbursed,DeductibleAmtPaid,Admitted,DurationofClaim,NumberofDaysAdmitted,RenalDiseaseIndicator,NoOfMonths_PartACov,NoOfMonths_PartBCov,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,AliveorDead,ClmDiagnosisCodeIndex,ClmProcedureCodeIndex,PerProviderAvg_InscClaimAmtReimbursed,PerProviderAvg_DeductibleAmtPaid,PerProviderAvg_IPAnnualReimbursementAmt,PerProviderAvg_IPAnnualDeductibleAmt,PerProviderAvg_OPAnnualReimbursementAmt,PerProviderAvg_OPAnnualDeductibleAmt,PerProviderAvg_Age,PerProviderAvg_NoOfMonths_PartACov,PerProviderAvg_NoOfMonths_PartBCov,PerProviderAvg_DurationofClaim,PerProviderAvg_NumberofDaysAdmitted,PerBeneIDAvg_InscClaimAmtReimbursed,PerAttendingPhysician Avg_InscClaimAmtReimbursed,PerBeneIDAvg_DeductibleAmtPaid,PerAttendingPhysician Avg_DeductibleAmtPaid,PerBeneIDAvg_IPAnnualReimbursementAmt,PerAttendingPhysician Avg_IPAnnualReimbursementAmt,PerBeneIDAvg_IPAnnualDeductibleAmt,PerAttendingPhysician Avg_IPAnnualDeductibleAmt,PerBeneIDAvg_OPAnnualReimbursementAmt,PerAttendingPhysician Avg_OPAnnualReimbursementAmt,PerBeneIDAvg_OPAnnualDeductibleAmt,PerAttendingPhysician Avg_OPAnnualDeductibleAmt,PerBeneIDAvg_DurationofClaim,PerAttendingPhysician Avg_DurationofClaim,PerBeneIDAvg_NumberofDaysAdmitted,PerAttendingPhysician Avg_NumberofDaysAdmitted,ChronicCond_Alzheimer_1,ChronicCond_Alzheimer_2,ChronicCond_Heartfailure_1,ChronicCond_Heartfailure_2,ChronicCond_KidneyDisease_1,ChronicCond_KidneyDisease_2,ChronicCond_Cancer_1,ChronicCond_Cancer_2,ChronicCond_ObstrPulmonary_1,ChronicCond_ObstrPulmonary_2,ChronicCond_Depression_1,ChronicCond_Depression_2,ChronicCond_Diabetes_1,ChronicCond_Diabetes_2,ChronicCond_IschemicHeart_1,ChronicCond_IschemicHeart_2,ChronicCond_Osteoporasis_1,ChronicCond_Osteoporasis_2,ChronicCond_rheumatoidarthritis_1,ChronicCond_rheumatoidarthritis_2,ChronicCond_stroke_1,ChronicCond_stroke_2
0,90,0.0,0,0,0.0,0,12,12,14000,2136,490,30,79.0,0,3,0,370.578616,19.376638,4437.509434,576.311950,2071.433962,617.283019,72.523270,11.969811,11.879245,1.262893,0.085535,2878.000000,423.840580,427.2,8.898551,14000.0,4633.043478,2136.0,464.347826,490.0,2078.188406,30.0,648.115942,1.2,0.731884,1.600000,0.050725,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,200,0.0,0,0,0.0,1,12,12,0,0,800,290,99.0,0,3,0,1121.590190,89.961119,5009.280063,501.743671,2116.946203,649.881329,73.796677,12.000000,11.973892,1.907437,0.545095,114.285714,201.415929,0.0,6.637168,0.0,4274.867257,0.0,406.407080,800.0,2229.734513,290.0,695.575221,0.0,1.389381,0.000000,0.000000,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
2,500,0.0,0,0,0.0,0,12,12,13000,1068,1900,580,84.0,0,6,0,2761.339713,288.595347,5953.971292,767.387560,1834.593301,552.488038,73.789474,12.000000,11.942584,2.215311,1.760766,2360.000000,2761.339713,178.0,288.595347,13000.0,5953.971292,1068.0,767.387560,1900.0,1834.593301,580.0,552.488038,5.5,2.215311,1.333333,1.760766,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,40,0.0,0,0,0.0,0,12,12,0,0,140,80,94.0,0,1,0,392.819549,21.879699,4817.857143,595.548872,2252.368421,686.390977,74.176692,11.909774,11.977444,1.308271,0.097744,30.000000,40.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,140.0,140.000000,80.0,80.000000,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,60,0.0,0,0,0.0,0,12,12,0,0,60,0,73.0,0,1,0,354.927536,22.183575,3167.729469,340.521739,1706.425121,635.314010,73.758454,11.942029,12.000000,1.483092,0.077295,60.000000,274.375000,0.0,1.562500,0.0,1218.750000,0.0,100.125000,60.0,1529.687500,0.0,575.000000,0.0,0.843750,0.000000,0.000000,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [45]:
X_train.to_csv('xtrain.csv', index= False, header= False)
X_test.to_csv('xtest.csv', index= False, header= False)
X_val.to_csv('xval.csv', index= False, header= False)

In [46]:
y_train.to_csv('ytrain.csv', index= False, header= False)
y_val.to_csv('yval.csv', index= False, header= False)

In [48]:
import pandas as pd
xtrain = pd.read_csv('xtrain.csv')
print(len(xtrain.columns))

65


In [49]:
ytrain, yval = pd.read_csv('ytrain.csv', header= None), pd.read_csv('yval.csv', header = None)
ytrain[0] = ytrain[0].map({'Yes': 1, 'No': 0})
yval[0] = yval[0].map({'Yes': 1, 'No': 0})
ytrain.to_csv('ytrain.csv', index= False, header= False)
yval.to_csv('yval.csv', index= False, header= False)


In [55]:
print(ytrain[0].value_counts())

0
1    310111
0    310111
Name: count, dtype: int64
